## 1. 模型评价

### 1.1 两类算法

在机器学习算法中有两类算法：有监督学习算法和无监督学习算法.

有监督学习是基于数据$(X, Y)$, 通过某个算法建立通过$\mathbf x$预测$y$, 或者对$y$分类的模型.
- 这里, $\mathbf x=(x_1,\dots,x_p)^{T}$是$p$维自变量, 或者输入变量集, $X$是$\mathbf x$的$n$个观测组成的
$n\times p$维矩阵;
- $Y$是自变量, 或者输出变量$y$的$n$个观测组成的$n$维向量矩阵, 当然也可以是$n\times 1$维矩阵;
- 如果$y$是连续性变量, 有监督学习问题是预测或者回归问题, 如果$y$是分类变量, 有监督学习问题是分类问题

### 1.2 模型评价标准: 模型的预测效果

本课程中不区分建模方法和算法. 有些算法非常简单, 有些算法则相对复杂. 
- 例如在回归问题中, 你认为所有的自变量都应该在模型中, 可以用最小二乘算法得到的全模型. 
- 而最优子集回归算法则认为自变量中会有冗余变量, 用最小二乘算法得到了$2^p$个模型.

无论上面的那种情况, 你都需要评价模型.
- 你把用最小二乘算法得到的全模型给你的客户, 你的客户会问你: 这个模型好吗?
- 你用最优子集回归算法得到$2^p$个模型, 你同样需要选择出“最优”的那个模型, 你的“最优”模型同样要接受你客户的“拷问”： 这个模型好吗?
本讲将以预测问题为题, 讨论算法中的模型评价问题.

既然是评价, 那么有个问题至关重要: 什么是标准? 
本节, 我们首先给出模型选择的标准.

- Breiman(2001)对传统数据建模中的诸多做法做了很深刻的反思,
并提出几个影响深远的观点, 其中之一即"应该用模型的预测效果作为模型选择或者验证的标准", 当然这并不是Leo
Breiman的首倡.

关于模型的预测效果有两个指标: 测试误差(Test Error)和期望测试误差(Expected Test Error).
-测试误差为
\begin{equation}
\mathrm{Err}_{\mathcal{T}}= \mathrm{E}_{\mathbf x, y}(L(\mathbf{y}, \hat{f}(\mathbf x))| \mathcal{T}),
\end{equation}
也称作推广误差(Generalized Error, 有人也译作泛化误差),
本课程沿用了Hastie, et al. (2009)的名称. 式(3.2)中, $L(\cdot)$是损失函数, $\mathcal{T}$是训练集,
$\hat{f}(x)$是通过训练集$\mathcal{T}$, 建立的对真实模型$f(\mathbf x)$的近似. 在回归问题中, 损失函数一般选择平方损失函数.

- 测试误差用来评价模型会个缺点：测试误差会出现小于训练训练误差的情况.

In [31]:
import numpy as np
from scipy import linalg
from scipy.stats import norm

def msig(p, pho):
    if p == 1:
        return np.array(np.arange(1,2,dtype=float)).reshape(1,1)
    else:
        tmp = np.hstack( (msig(p-1, pho), np.power(pho*np.ones(p-1), np.arange(p-1, 0, -1)).reshape(p-1, 1)) )
        return np.vstack( (tmp,  np.power(pho*np.ones(p), np.arange(p-1,-1,-1))) )
def solve_sym(xtx, xty):
        L = linalg.cholesky(xtx)
        return linalg.lapack.dpotrs(L, xty)[0]

import random
random.seed(111)
p = 10
n = 100
rho = 0.5
mu = 1
beta = np.ones(p)


def TestandTrain_Err(n=n, p=p, rho=rho, mu=mu, beta=beta):
    z = norm.rvs(size=(n, p))

    V = msig(p, rho)
    L = linalg.cholesky(V)
    x = np.dot(z, L)
    fx = mu + np.dot(x, beta)
    Y = fx + 5*norm.rvs(size=n)  #训练集
    YS = fx + 5*norm.rvs(size=n) #测试集
    
    X = np.c_[np.ones((n,1)), x]
    XT = X.T
    XTX = np.dot(XT, X)
    XTY = np.dot(XT, Y)
    b = solve_sym(XTX, XTY)
    YH = np.dot(X, b)
    Train_Err = np.mean((Y-YH)*(Y-YH))
    Test_Err = np.mean((YS-YH)*(YS-YH))
    return Test_Err-Train_Err
re = [TestandTrain_Err(n, p, rho, mu, beta) for _ in np.arange(1000)] 
np.mean( np.array(re) <=0 )

0.149

In [32]:
np.mean(re)

5.275580191525018

假定的模型是一个有10个自变量的模型, 训练集和测试集的样本量都是100, 另外, 在每次
模拟中, 自变量的观测是固定的. 最后的模拟结果显示, 近15%的模拟中, 训练集误差是小于测试集误差的;
而测试集误差与训练集误差的平均则是大于0的. 很自然地, 考虑用测试误差的期望来衡量模型的预测效果
更经得起检验, 具体地,
期望测试误差为
\begin{equation}
\mathrm{Err}= \mathrm{E}_{\mathcal T}(\mathrm{Err}_{\mathcal{T}}).
\end{equation}
事实上期望测试误差是对模型``稳定性"的考虑,
如果换一个训练数据, 得到大相径庭的结果, 那么这个模型可信吗?

## 2. 模型评价指标

### 2.1 $C_p$和AIC

Hastie, et al. (2009)定义了这样一个类似测试误差的指标: “样本内”(in-sample)测试误差
\begin{equation}
\mathrm{Err_{in}} = \frac 1 n \sum_{i=1}^n \mathrm{E}_{y_i^*}\left[(y_i^*- \hat{y}_i| Y\right],
\end{equation}
其中, $y_i^*$是在式(3.6)的设定下, 区别于训练集中$y_i$的另一随机实现,
$\hat{y}_i$是第$i$个样本的拟合值.
之所以称$\mathrm{Err_{in}}$为“样本内”(in-sample)期望测试误差, 是因为也就是基于样本中自变量$\mathbf x$的实现$X$构造的测试误差.
事实上, 如果自变量$\mathrm x$是随机变量, 当$X$的代表性足够好时, $\mathrm{Err_{in}}$就是测试误差.

- 很自然, 我们可以用的期望“样本内”(in-sample)测试误差代替期望测试误差, 而且还有下面的分解.
\begin{equation}
\mathrm{E}_{Y}(\mathrm{Err_{in}}) = \mathrm{E}_{Y} (\mathrm{err})
+\frac 2 n \sum_{i=1}^ n \mathrm{Cov}(y_i, \hat{y}_i),
\end{equation}
其中,
\begin{equation}
\mathrm{err} = \frac 1 n \sum_{i=1}^n (y_i- \hat{f}(\mathrm x_i))^2,
\end{equation}
$\hat{y}_i=\hat{f}(\mathbf x_i).$
上式等号右边的两项也是此消彼长的关系, 模型拟合效果越好, 也就是$\mathbf{E}_{Y} (\mathrm{err})$越小,
自然$\sum_{i=1}^ n \mathrm{Cov}(y_i, \hat{y}_i)$就越大.

记$\hat{Y}=(\hat{f}(\mathbf x_1),\dots,\hat{f}(\mathbf x_n))^T,$ 假定$\hat{ Y}=S Y$(线性拟合),
那么,
$$\sum_{i=1}^ n \mathrm{Cov}(y_i, \hat{y}_i) = \mathrm{trac}(S)\sigma^2.$$
对于线性回归模型, $\mathrm{trac}(S)=d,$ $d$是线性模型系数的个数(含截距项).
于是, 式(3.8)为
\begin{equation}
\mathrm{E}_{Y}(\mathrm{Err_{in}}) = \mathrm{E}_{ Y} (\mathrm{err})
+\frac 2 n d\sigma^2,
\end{equation}

如果用$\mathrm{err}$估计$\mathrm{E}_{ Y} (\mathrm{err}),$
使用全模型得到$\sigma^2$的无偏估计$\hat{\sigma}^2,$ 就可以得到
$\mathrm{E}_{Y}(\mathrm{Err_{in}})$的估计, 也就是$C_p$统计量
\begin{equation}
C_p =  \mathrm{err} +\frac 2 n d \hat{\sigma}^2.
\end{equation}

很显然, $\mathrm{err}$越小, $d$就越大.
用上式解释“过拟合”或许更加贴切, 以最优子集回归为例, 随着自变量个数$d$的增加,
当$\mathrm{err}$的减小量不能抵消$d$的增大带来的“坏处”时, $C_p$统计量就开始变大了.
所以, “过拟合”其实是过于看重拟合效果, 导致模型的预测效果变差了.

- 在普通回归中, 还有类似的指标AIC
\begin{equation}
\mathrm{AIC}=n\log RSS +2p +\mathrm{constant},
\end{equation}
其中RSS为残差平方和

### 2.2 交叉验证

- 把数据分成$K$折.
- 依次去掉第$k$折数据, 训练出模型, 给出在第$k$折数据的预测值, $k=1,\dots,K$
- 构建$K$折交叉验证误差
$$ \frac 1 n  \sum_{k=1}^K \left(y_{ki} - \hat{y}_{ki}^{(-k)}\right)^2$$
- $K$一般取值为10

## 课堂练习：请大家编写$C_p$, $AIC$和交叉验证作为准则的最优子集回归

In [ ]:
import numpy as np
from scipy import linalg
def turnbits_rec(p):
    if(p==1):
        return np.array([[True, False],[True, True]])
    else:
        tmp1 = np.c_[ turnbits_rec(p-1), np.array([False]*(2**(p-1))).reshape((2**(p-1),1))]
        tmp2 = np.c_[ turnbits_rec(p-1), np.array([True]*(2**(p-1))).reshape((2**(p-1),1))]
        return np.r_[tmp1, tmp2]

def solve_sym(xwx, s):
    L = linalg.cholesky(xwx)
    return linalg.lapack.dpotrs(L, s)[0]

class BestSubsetReg(object):
    def __init__(self, x = 0, y = 0, inter = True, isCp = True, isAIC = True, isCV = True):
        self.n, self.p = x.shape
        if inter:
            self.x = np.c_((np.ones((self.n, 1), x))
        else:
            self.x = x
        self.y = y
        self.xx = np.dot(self.x.T, self.x)
        self.xy = np.dot(self.x.T, self.y)
        self.ind_var = turnbits_rec(p)
        self.b = []
        if isCp:
            self.Cp = 0
        if isAIC:
            self.AIC = 0
        if isCV:
            self.cverr = 0
                           
                           
    def reg(self):
        self.b = [solve_sym(self.xtx[ind][:,ind], self.xty[ind]) for ind in self.ind_var]
    def Cp_AIC(self):
        mse_tmp = [ np.sum(np.dot(xtx[ind][:,ind], b_)*b_) for ind,b_ in zip(self.ind_var, self.b)]
        rss = np.sum(self.y*self.y) - mse_tmp
        d = np.sum(self.ind_var, axis = 0)
        if isCp:
            self.Cp = rss/rss[-1]*(self.n - self.p - 1) + 2*d
        if isAIC:
            self.AIC = self.n*np.log(rss) + 2*d
                           
    def cvreg(self)：
        k = 10
        indexs = np.array_split(np.random.permutation(np.arange(0, self.n)), k)
        def cvk(ind, index):
            txx = self.xx[ind][:,ind] - np.dot((self.x[index][:,ind]).T, self.x[index][:,ind])
            txy = self.xy[ind] - np.dot((self.x[index][:,ind]).T, self.y[index])
            tcoe = solve_sym(txx, txy)
            return np.sum((self.y[index] - np.dot(self.x[index][:,ind], tcoe))**2)
        self.cverr = np.sum(np.array([cvk(ind, index) for ind in self.ind_var for index in indexs]), axis = 0)/self.n